# Exercise 6 - Optimal offering strategy of price-taker wind producer

## Imports

In [2]:
from gurobipy import Model, GRB

## Statement

In [4]:
prices = [10, 10, 30, 30]  # EUR/MWh
productions = [125, 75, 125, 75]  # MWh
probabilities = [0.25, 0.25, 0.25, 0.25]  # Probabilities for each scenario
marginal_cost = 15  # EUR/MWh

## Resolution

### a)

The possible scenarios for price and production are as follows:
   - $\(\lambda = 10 \, \text{EUR/MWh}, P_{W1} = 125 \, \text{MWh} \) (Probability = 0.25)$
   - $\(\lambda = 10 \, \text{EUR/MWh}, P_{W1} = 75 \, \text{MWh} \) (Probability = 0.25)$
   - $\(\lambda = 30 \, \text{EUR/MWh}, P_{W1} = 125 \, \text{MWh} \) (Probability = 0.25)$
   - $\(\lambda = 30 \, \text{EUR/MWh}, P_{W1} = 75 \, \text{MWh} \) (Probability = 0.25)$

### b)

The wind producer decides the amount of electricity to offer before knowing the actual price or wind production level. Once the offer is made, it can only be changed in the intradaya and balancing markets

### c)

The objective of the wind producer is to maximize expected profit. The profit for each scenario is given by:

$
\text{Profit} = (\lambda - 15) \times q
$

Where:
- $q$ is the quantity of electricity offered in the day-ahead market.
- $\lambda$ is the electricity price.
- 15 EUR/MWh is the marginal cost of production.

The expected profit over all scenarios can be written as:

$
\text{Expected Profit} = 0.25 \times \left[ (\lambda_1 - 15) \times q_1 + (\lambda_2 - 15) \times q_2 + \dots \right]
$

Subject to wind production constraints for each scenario.

### d)

For each scenario, the wind producer can only produce up to the available wind production

### e)

In [5]:
model = Model('WindProducer')

# Decision variable: q (offered electricity)
q = model.addVar(lb=0, name="q")  # Amount of electricity offered in the market

# Objective function: Maximize expected profit
expected_profit = sum(probabilities[i] * (prices[i] - marginal_cost) * q for i in range(4))
model.setObjective(expected_profit, GRB.MAXIMIZE)

# Constraints: Wind production limit for each scenario
for i in range(4):
    model.addConstr(q <= productions[i], f"Production_limit_scenario_{i}")

# Solve the model
model.optimize()

# Output the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal amount of electricity to offer: {q.x:.2f} MWh")
    print(f"Expected profit: {model.objVal:.2f} EUR")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10700T CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 1 columns and 4 nonzeros
Model fingerprint: 0xdc0b52c2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 1e+02]
Presolve removed 4 rows and 1 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.750000000e+02
Optimal amount of electricity to offer: 75.00 MWh
Expected profit: 375.00 EUR
